In [11]:
import torch
import torch.nn as nn
#import torchvision.models as models
import torch.nn.functional as F
import numpy as np

print(torch.__version__)

1.3.0


## 依照指示取出模型特定層的資訊

In [12]:
# Define model
class TheModelClass(nn.Module):
  def __init__(self):
      super(TheModelClass, self).__init__()
      self.conv1 = nn.Conv2d(3, 6, 5)
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(6, 16, 5)
      self.fc1 = nn.Linear(16* 5 * 5, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 16 * 5 * 5)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [13]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [14]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [15]:
outputs

[tensor([[[[0.8510, 0.2265, 0.0409,  ..., 0.5597, 0.7790, 0.7186],
           [0.3253, 0.5405, 0.7610,  ..., 1.3477, 0.0000, 0.1665],
           [0.2214, 0.5064, 1.3086,  ..., 0.6682, 0.8030, 0.7806],
           ...,
           [0.9789, 0.5931, 1.2260,  ..., 0.8235, 0.9098, 1.3077],
           [0.6781, 0.6227, 0.8199,  ..., 0.1440, 1.3091, 1.5261],
           [1.1781, 0.8866, 0.3521,  ..., 0.1075, 0.5348, 0.6083]],
 
          [[0.2995, 0.2500, 0.1462,  ..., 0.2405, 0.4472, 0.5872],
           [0.1748, 0.6589, 0.2263,  ..., 0.4175, 0.4352, 0.0000],
           [0.0000, 0.9496, 0.0000,  ..., 1.5859, 0.3300, 0.0000],
           ...,
           [0.3426, 0.9699, 0.9020,  ..., 0.6494, 1.3301, 1.0731],
           [0.0000, 0.3457, 1.1749,  ..., 0.9399, 0.5258, 0.7047],
           [0.5868, 0.5206, 0.5198,  ..., 0.0807, 0.6264, 1.1088]],
 
          [[0.2042, 0.5585, 0.6820,  ..., 0.1754, 0.0129, 0.3294],
           [0.2808, 0.6039, 0.4003,  ..., 0.8143, 0.3895, 0.7608],
           [0.6985, 1.15

In [16]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [17]:
from torch.nn import init

def weights_init(m):
  #classname = m.__class__.__name__
  if isinstance(m, nn.Conv2d):
    #torch.nn.init.xavier_uniform_(m.weight)
    nn.init.kaiming_normal_(m.weight.data,
                a=0,
                mode='fan_out',
                nonlinearity='relu')
    if m.bias is not None:
        m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [18]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


